# Fund -  fund value next to project max outcome payments

## Libs and setup


In [ ]:
!pip install --upgrade plotly
!pip install kaleido

In [ ]:
import requests
import plotly.graph_objects as go



INDIGO_DATABASE_API = 'https://golab-indigo-data-store.herokuapp.com/app/api1'


def api_get_item(endpoint, public_id=None):
    """
    Get individual item details from the API

    E.g. 
    item = api_get_item('project', 'INDIGO-POJ-0158')
    """
    try:
        if public_id:
            response = requests.get(f'{INDIGO_DATABASE_API}/{endpoint}/{public_id}')
        else:
            response = requests.get(f'{INDIGO_DATABASE_API}/{endpoint}')
        item = response.json()
        return item
    except Exception as e:
        print(f'\nFailed to retrieve {endpoint} "{public_id}".\nError: {e}')
        return False


def viz_for_fund(fund_id, fund_project_list, fund_data, filename=None, yaxis_range=None):
    data = []
    viz_currencies = set()
    for project_id in fund_project_list:
        project_amount = project_maximum_potential_outcome_payments.get(project_id,{}).get('amount')
        project_amount = float(project_amount) if project_amount else 0
        project_currency = project_maximum_potential_outcome_payments.get(project_id,{}).get('currency')
        if project_amount and project_currency:
            data.append(go.Bar(name=project_id, x=['Projects'], y=[project_amount]))
            viz_currencies.add(project_currency)
    fund_value_data = fund_data.get('dimensions_of_scale', {}).get('total_outcome_fund_value_outcomes_funding',{})
    fund_value_amounts = [
        fund_value_data.get('actual',{}).get('amount',{}).get('value',{}),    
        fund_value_data.get('aspiration',{}).get('amount',{}).get('value',{}),              
    ]
    fund_value_amounts = [i for i in fund_value_amounts if i]

    if fund_value_amounts:
        viz_currencies.add(fund_value_data.get('currency',{}).get('value',{}))
        data.append(go.Bar(name=fund_id, x=['Fund'], y=[fund_value_amounts[0]]))
    if data:
        if len(viz_currencies) > 1:
            print("Can not draw as mix of currencies")
            print(viz_currencies)
        else:
            fig = go.Figure(data=data)
            fig.update_layout(barmode='stack', yaxis_range=yaxis_range, yaxis_title=list(viz_currencies)[0])
            fig.show()
            if filename:
                fig.write_image(filename)

def vis_for_all_funds_in_local_currency(fund_ids, local_currency, filename=None, width=1000, height=1000):
    data_x = []
    data_y = []
    for fund_id in fund_ids:
        fund_value_data = fund_data[fund_id].get('dimensions_of_scale', {}).get('total_outcome_fund_value_outcomes_funding',{})
        fund_value_amounts = [
            fund_value_data.get('actual',{}).get('amount',{}).get('value',{}),    
            fund_value_data.get('aspiration',{}).get('amount',{}).get('value',{}),              
        ]
        fund_value_amounts = [i for i in fund_value_amounts if i]
        fund_currency = fund_value_data.get('currency',{}).get('value',{})
        fund_name = fund_data[fund_id].get('name', {}).get('value')
        if fund_value_amounts and fund_currency == local_currency:
            data_x.append(fund_id+": "+ fund_name)
            data_y.append(fund_value_amounts[0])
    if data_x:
            fig = go.Figure(data=[go.Bar(name="Total Outcome Fund value", x=data_x, y=data_y)])
            fig.update_layout(yaxis_title=local_currency, width=width, height=height)
            fig.show()
            if filename:
                fig.write_image(filename)


def vis_for_all_funds_in_usd(fund_ids, filename=None, width=1000, height=1000):
    data_x = []
    data_y = []
    for fund_id in fund_ids:
        fund_value_data = fund_data[fund_id].get('dimensions_of_scale', {}).get('total_outcome_fund_value_outcomes_funding',{})
        fund_value_amounts = [
            fund_value_data.get('actual',{}).get('amount_usd',{}).get('value',{}),    
            fund_value_data.get('aspiration',{}).get('amount_usd',{}).get('value',{}),              
        ]
        fund_value_amounts = [i for i in fund_value_amounts if i]
        fund_name = fund_data[fund_id].get('name', {}).get('value')
        if fund_value_amounts:
            data_x.append(fund_id+": "+ fund_name)
            data_y.append(fund_value_amounts[0])
    if data_x:
            fig = go.Figure(data=[go.Bar(name="Total Outcome Fund value", x=data_x, y=data_y)])
            fig.update_layout(yaxis_title='USD', width=width, height=height)
            fig.show()
            if filename:
                fig.write_image(filename)



## Get Fund data
Call the INDIGO API 'fund' endpoint and retrieve the data used for the plot.

By default this will get data for all the funds, but you can pass a list of fund ID's and get only some. See the comments in the code.

In [ ]:
# Call the API and pull down the data for each fund
# and store in a fund for use later.
# 
# You can set public_ids to some funds only
# eg:
# public_ids = ['INDIGO-FUND-0001', ]
# or pass an empty list, in which case it will get data from all funds
# eg:
# public_ids = []

public_ids = []
endpoint = 'fund'

if not public_ids:
    for fund_data in api_get_item(endpoint).get('funds'):
        if fund_data.get('public'):
            public_ids.append(fund_data.get('id'))

fund_data = {}

for fund_id in public_ids:
    print("Getting Fund " + fund_id)
    fund_data[fund_id] = api_get_item(endpoint, fund_id).get('fund',{}).get('data',{})

## Get Project Data

In [ ]:
!wget https://golab-indigo-data-store.herokuapp.com/app/all_public_data_file_per_data_type_csv.zip
!unzip -o all_public_data_file_per_data_type_csv.zip

In [ ]:
import csv
from collections import defaultdict

# Get a list of which funds have which projects
fund_has_projects = defaultdict(list)
with open('projects_outcome_funds.csv', newline='') as csvfile:
     outcomefundreader = csv.DictReader(csvfile)
     for row in outcomefundreader:
         fund_has_projects[row['Id']].append(row['Project ID'])

# Get a list of Project Funding Data
project_maximum_potential_outcome_payments =  {}
with open('projects.csv', newline='') as csvfile:
     outcomefundreader = csv.DictReader(csvfile)
     for row in outcomefundreader:
         project_maximum_potential_outcome_payments[row['ID']] = {
             'amount': row['Overall project finance - Maximum potential outcome payment - Amount - (Value)'],
             'amount_usd': row['Overall project finance - Maximum potential outcome payment - Amount USD - (Value)'],
             'currency': row['Overall project finance - Maximum potential outcome payment - Currency - (Value)'],
         }


## For every fund, an individual graph (Y axis varies)

In [ ]:
for fund_id, fund_project_list in fund_has_projects.items():
    print("Fund " + fund_id)
    viz_for_fund(fund_id, fund_project_list, fund_data[fund_id], filename='fund-'+fund_id+'-varies.png')


## For every fund, an individual graph (Y axis fixed)

In [ ]:
for fund_id, fund_project_list in fund_has_projects.items():
    print("Fund " + fund_id)
    viz_for_fund(fund_id, fund_project_list, fund_data[fund_id], filename='fund-'+fund_id+'-fixed.png', yaxis_range=[0,150000000])


## For all funds, vis

In [ ]:
vis_for_all_funds_in_local_currency(fund_data.keys(), 'GBP',filename='all-funds-gbp.png')
vis_for_all_funds_in_usd(fund_data.keys(),filename='all-funds-usd.png')